In [ ]:
# install
using Pkg

Pkg.add("NLsolve")
Pkg.add("Optim")

In [1]:
using NLsolve

A graph is usualy given by the list of links [(source, target), ...]. However, for the particular type of graph, chains of nodes are considered. So the graph is given by chains of nodes: [[A1, A2, ...], ...].

In [2]:
graph = [["074559265", "033293546", "177194634", "132327031", "05942608X", "147936594",
        "194946746", "20308523X", "160289750", "103659625", "103664262", "07456501X",
        "074559192", "074576127", "113270321", "123899249", "122925262", "121644545",
        "122935314", "13235103X", "149658192", "149531125", "151099103", "168078465",
        "17817310X", "18105938X", "181232529", "195092317", "202363856"],
    ["069862753", "033922020", "185417396", "185923089", "161170161", "188021574",
        "188160396", "188705945", "189504048", "189565624", "189997680", "190237201",
        "06986263X", "074589644", "076964361", "076694348", "076969576", "083593217",
        "110189159", "129545279"], ["112512380", "109209265", "185124100", "180818260",
        "185775926", "185887015", "187849390", "18786425X", "188021574", "112508901"],
    ["129545678", "129545279", "140093052", "154959588", "155289756", "176736999",
        "177538619", "179791338", "177523824", "195092317"],
    ["177539135", "177538619", "200874586", "241318939"],
    ["074576127", "168078465", "181232529"], ["188160396", "236926446"]];

In [16]:
graph = [["one", "two", "three"],
         ["two", "four", "five"]];

In [17]:
nodes = Dict(n=>rand(Float32, 2) for ch in graph for n in ch);
println("#nodes: ", length(nodes))
println("#chains: ", length(graph))

#nodes: 5
#chains: 2


Next we need a function which compute the system energy (and the forces?) for a given graph and a list of nodes coordinates. Folowing classical approach [Fruchterman and Reingold 1991](http://www.mathe2.uni-bayreuth.de/axel/papers/reingold:graph_drawing_by_force_directed_placement.pdf) there is an attractive force -between linked nodes- and repulsive forces between every nodes. In addition, here we want a torque at each node in a chain to rigidify the line. Maybe, a global gravity field would be required to densify the graph (keep disconected parts). 

$$
f_{attractive} = d^2/k \\
f_{repulsive} = -k^2/d
$$

In [18]:
norm(x) = sqrt(sum(x.^2));

In [20]:
const k = .1 # whatever the unit

function attractive_energy(x1, x2)
    d3 = norm(x1 .- x2).^3
    E = d3/k/3
    end;

function repulsive_energy(x1, x2)
    d = norm(x1 .- x2)
    E = -k^2*log(d)
    end;

#println( attractive_energy(nodes["033293546"], nodes["177194634"]) )
#println( repulsive_energy(nodes["033293546"], nodes["177194634"]) )

The torque will be function of the angle formed between 3 ordered nodes. 

In [22]:
function join_angle(x1, x2, x3)
    x12 = x2 .- x1
    x23 = x3 .- x2
    dot = sum( x12 .* x23 )/(norm(x12)*norm(x23))
    theta = acos(dot)
    end;

function join_energy(x1, x2, x3)
    theta = join_angle(x1, x2, x3)
    gamma = .008 # J/rad2
    E = gamma*theta^2
    end;

#x1, x2, x3 = nodes["033293546"], nodes["177194634"], nodes["132327031"]
#println(join_angle(x1, x2, x3))
#println(join_energy(x1, x2, x3))

In [23]:
function isolated_chain_energy(chain, nodes)
    nrj = 0
    for k in 1:length(chain)-2
        n1, n2, n3 = chain[k], chain[k+1], chain[k+2]
        x1, x2, x3 = nodes[n1], nodes[n2], nodes[n3]
        nrj += join_energy(x1, x2, x3)
        nrj += attractive_energy(x1, x2)
        end;
    # last link:
    nrj += attractive_energy(nodes[chain[end-1]], nodes[chain[end]])
    end;

function global_chain_energy(graph, nodes)
    nrj = 0
    for chain in graph
        nrj += isolated_chain_energy(chain, nodes)
        end;
    return nrj
    end;

In [25]:
chain = graph[2]
isolated_chain_energy(chain, nodes)

2.038097500324249

In [26]:
global_chain_energy(graph, nodes)

2.3307935562779507

In [27]:
function global_repulsive_energy(nodes)
    nrj = 0
    for (n1, x1) in pairs(nodes)
        for (n2, x2) in pairs(nodes)
            if n1 > n2
                nrj += repulsive_energy(x1, x2)
                end;
            end;
        end;
    return nrj
    end;

println(global_repulsive_energy(nodes))

function graph_energy(graph, nodes)
    nrj = global_repulsive_energy(nodes)
    nrj += isolated_chain_energy(chain, nodes)
    return nrj
    end;

0.0998519751429558


In [28]:
graph_energy(graph, nodes)

2.137949475467205

Let's now try to solve this mess

In [29]:
nodes_list = String[]
coords = Float32[]
for (n, x) in pairs(nodes)
    push!(nodes_list, n)
    append!(coords, x)
    end;

In [36]:
function proxy_function(x)
    for (i, n) in enumerate(nodes_list)
        nodes[n] = [x[2*i-1], x[2*i]] 
        end;
    E = graph_energy(graph, nodes)
    end;

In [37]:
proxy_function(coords)

2.137949475467205

In [39]:
nlsolve(proxy_function, coords, iterations=3)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: Float32[0.6238532, 0.36713898, 0.7998136, 0.8389406, 0.78978837, 0.48203838, 0.90661144, 0.63168323, 0.41286337, 0.16936028]
 * Zero: Float32[NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 * Inf-norm of residuals: 2.137949
 * Iterations: 3
 * Convergence: false
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: false
 * Function Calls (f): 4
 * Jacobian Calls (df/dx): 1